In [6]:
import rope.base.project
from rope.refactor.extract import ExtractMethod
import re

APOSTROPHE_MARKER = "__APOSTROPHE__"
PERCENT_FORMAT_MARKER = "__PERCENT_FORMAT__"

def serialize(text: str):
    # Replace "'{var}'" with "__APOSTROPHE__{var}__APOSTROPHE__"
    text = re.sub(r"'{([^'}]*?)}'", f"{APOSTROPHE_MARKER}{{\\1}}{APOSTROPHE_MARKER}", text)
    # Replace "%s" with "__PERCENT_FORMAT__"
    text = re.sub(r"%\((.*?)\)s", f"{PERCENT_FORMAT_MARKER}{{\\1}}", text)
    return text

def deserialize(text: str):
    text = re.sub(f"{APOSTROPHE_MARKER}{{(.*?)}}{APOSTROPHE_MARKER}", "'{\\1}'", text)
    text = re.sub(f"{PERCENT_FORMAT_MARKER}{{(.*?)}}", "%(\\1)s", text)
    return text

myproject = rope.base.project.Project('../../')

myresource = myproject.get_resource('tests/notebooks/src/test.py')
contents = myresource.read()
serialized_contents = serialize(myresource.read())
myresource.write(serialized_contents)
extract_span = r"""if (
                MULTI_REGION_CONFIG is None
                or not isinstance(MULTI_REGION_CONFIG, list)
                or len(MULTI_REGION_CONFIG) == 0
                or not isinstance(MULTI_REGION_CONFIG[0], list)
            ):
                logger.info(
                    f"Calling {model} with engine {engine} on Azure url {OPENAI_API_BASE}."
                )
                openai.api_type = OPENAI_API_TYPE
                openai.api_base = OPENAI_API_BASE
                openai.api_version = OPENAI_API_VERSION
                openai.api_key = AZURE_API_KEY"""
serialized_extract_span = serialize(extract_span)
print(serialized_extract_span)

start, end = serialized_contents.find(serialized_extract_span), serialized_contents.find(serialized_extract_span) + len(serialized_extract_span)
print(start, end)

try:
    extractor = ExtractMethod(myproject, myresource, start, end)
    change_set = extractor.get_changes("helper", similar=True)
    for change in change_set.changes:
        if change.old_contents is not None:
            change.old_contents = deserialize(change.old_contents)
        else:
            change.old_contents = deserialize(change.resource.read())
        change.new_contents = deserialize(change.new_contents)
    for change in change_set.changes:
        print(change.get_description())
except Exception as e:
    print(e)
finally:
    myresource.write(contents)

engine = None
            if model in OPENAI_EXCLUSIVE_MODELS and OPENAI_API_TYPE != "azure":
                logger.info(f"Calling OpenAI exclusive model. {model}")
                raise Exception("OpenAI exclusive model.")
            if (
                model == "gpt-3.5-turbo-16k"
                or model == "gpt-3.5-turbo-16k-0613"
                and OPENAI_API_ENGINE_GPT35 is not None
            ):
                engine = OPENAI_API_ENGINE_GPT35
            elif (
                model == "gpt-4"
                or model == "gpt-4-0613"
                and OPENAI_API_ENGINE_GPT4 is not None
            ):
                engine = OPENAI_API_ENGINE_GPT4
            elif (
                model == "gpt-4-32k"
                or model == "gpt-4-32k-0613"
                and OPENAI_API_ENGINE_GPT4_32K is not None
            ):
                engine = OPENAI_API_ENGINE_GPT4_32K
767 1664
--- a/tests/notebooks/src/test.py
+++ b/tests/notebooks/src/test.py
@@ -37,28 +37,7 @@
     @

In [22]:
len(change.new_contents) - len(change.old_contents)

85